In [1]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm 
import os
from datasets import load_dataset, DatasetDict
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel,ViTModel,VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

In [2]:
vizwiz = load_dataset("Multimodal-Fatima/VizWiz_train")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
train_test_split = vizwiz['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

train_valid_split = train_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_valid_split["train"]
valid_dataset = train_valid_split["test"]

split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'image', 'filename', 'question', 'answers', 'answers_original', 'answer_type', 'answerable', 'id_image', 'clip_tags_ViT_L_14', 'clip_tags_LAION_ViT_H_14_2B', 'blip_caption_beam_5', 'LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14', 'LLM_Description_gpt3_downstream_tasks_visual_genome_LAION-ViT-H-14-2B', 'DETA_detections_deta_swin_large_o365_coco_classes'],
        num_rows: 14776
    })
    validation: Dataset({
        features: ['id', 'image', 'filename', 'question', 'answers', 'answers_original', 'answer_type', 'answerable', 'id_image', 'clip_tags_ViT_L_14', 'clip_tags_LAION_ViT_H_14_2B', 'blip_caption_beam_5', 'LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14', 'LLM_Description_gpt3_downstream_tasks_visual_genome_LAION-ViT-H-14-2B', 'DETA_detections_deta_swin_large_o365_coco_classes'],
        num_rows: 1642
    })
    test: Dataset({
        features: ['id', 'image', 'filename', 'question', 'answe

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

In [9]:
train_dataset = split_dataset['train'].remove_columns([
    'id', 'filename', 'answers_original', 'answer_type', 'answerable','id_image', 'clip_tags_ViT_L_14', 'clip_tags_LAION_ViT_H_14_2B',
    'blip_caption_beam_5', 
    'LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14', 
    'LLM_Description_gpt3_downstream_tasks_visual_genome_LAION-ViT-H-14-2B', 
    'DETA_detections_deta_swin_large_o365_coco_classes'
])


In [10]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [11]:
def image_captions(data):

    ''' Preprocesses the data by adding captions to the images.

        Arguments : 
        -data : a dict 

        Outputs : 
            An upgraded dict with the added captions for each image.
    '''
    images = []
    images.append(data['image'].convert("RGB"))
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    gen_kwargs = {"max_length": 16, "num_beams": 4}
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    captions = [pred.strip() for pred in preds]
    images = transform(data['image'].convert("RGB"))
    question = data['question']
    answers = data['answers'][0] 
    

    return {'image' : images,
           'question': question,
            'answers': answers,
            'captions': captions
           }

dataset_dict = train_dataset.map(preprocess_images)
pytorch_train = dataset_dict.with_format(type='torch')



Map:   0%|          | 0/14776 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [38]:
pytorch_train[34]

{'image': tensor([[[0.2353, 0.2157, 0.2078,  ..., 0.2941, 0.2824, 0.2824],
          [0.2471, 0.2392, 0.2235,  ..., 0.2902, 0.2863, 0.2902],
          [0.2588, 0.2588, 0.2706,  ..., 0.2902, 0.2941, 0.2902],
          ...,
          [0.1333, 0.1294, 0.1294,  ..., 1.0000, 1.0000, 1.0000],
          [0.1294, 0.1216, 0.1176,  ..., 1.0000, 1.0000, 1.0000],
          [0.1294, 0.1176, 0.1137,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.1725, 0.1216, 0.1098,  ..., 0.1843, 0.1725, 0.1725],
          [0.2314, 0.1882, 0.1333,  ..., 0.1804, 0.1765, 0.1804],
          [0.2392, 0.2392, 0.2196,  ..., 0.1804, 0.1804, 0.1804],
          ...,
          [0.0667, 0.0627, 0.0627,  ..., 1.0000, 1.0000, 1.0000],
          [0.0627, 0.0549, 0.0510,  ..., 1.0000, 1.0000, 1.0000],
          [0.0627, 0.0510, 0.0471,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.1490, 0.1020, 0.0863,  ..., 0.1294, 0.1176, 0.1176],
          [0.2039, 0.1647, 0.1137,  ..., 0.1255, 0.1216, 0.1255],
          [0.2196, 0.2157, 0.20

In [14]:
valid = split_dataset['validation'].remove_columns([
    'id', 'filename', 'answers_original', 'answer_type', 'answerable','id_image', 'clip_tags_ViT_L_14', 'clip_tags_LAION_ViT_H_14_2B',
    'blip_caption_beam_5', 
    'LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14', 
    'LLM_Description_gpt3_downstream_tasks_visual_genome_LAION-ViT-H-14-2B', 
    'DETA_detections_deta_swin_large_o365_coco_classes'
])


In [15]:
dataset_dict = valid.map(preprocess_images)
pytorch_valid = dataset_dict.with_format(type='torch')

Map:   0%|          | 0/1642 [00:00<?, ? examples/s]

In [45]:
class MultimodalVQAModel(nn.Module):
    def __init__(self):
        super(MultimodalVQAModel, self).__init__()
        
        # Image Encoder 
        self.vision_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224")
        self.vision_fc = nn.Linear(768, 512)  
        
        # Encoder for Question
        self.question_encoder = BertModel.from_pretrained("bert-base-uncased")
        self.question_fc = nn.Linear(768, 512)

        # Encoder for Caption 
        self.caption_encoder = BertModel.from_pretrained("bert-base-uncased")
        self.caption_fc = nn.Linear(768, 512)

        self.projection_fc = nn.Linear(1536, 768)

        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = tokenizer.eos_token 

        # Answer Generator 
        self.answer_decoder = GPT2LMHeadModel.from_pretrained("gpt2")


    def forward(self, image, questions, captions,answers):
        # Encode the image
        image_features = self.vision_encoder(image).pooler_output
        image_features = self.vision_fc(image_features)

        # Encode the question
        question_outputs = self.question_encoder(input_ids=questions['input_ids'], attention_mask=questions['attention_mask'])
        question_features = self.question_fc(question_outputs.pooler_output)#.to(device))

        # Encode the caption
        caption_outputs = self.caption_encoder(input_ids=captions['input_ids'], attention_mask=captions['attention_mask'])
        caption_features = self.caption_fc(caption_outputs.pooler_output)#.to(device))
        

        #print(f"Image Features Shape: {image_features.shape}")
        #print(f"Question Features Shape: {question_features.shape}")
        #print(f"Caption Features Shape: {caption_features.shape}")

        # Concatenate and fuse features
        combined_features = torch.cat((image_features, question_features, caption_features), dim=1)
        combined_features = self.projection_fc(combined_features)#.to(device))
        
        answer_input_ids = self.tokenizer(answers, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
        #print(answer_input_ids)

        # Reshape combined features for GPT-2 input
        sequence_length = answer_input_ids.size(1)  
        combined_features = combined_features.unsqueeze(1).repeat(1, sequence_length, 1)  # Shape: (batch_size, sequence_length, 768)
        #print(combined_features)
        
        
        # Generate output using the combined context with actual answers as labels
        outputs = self.answer_decoder(inputs_embeds=combined_features, labels=answer_input_ids)  # Use actual answers for training
        

        return outputs.loss, outputs.logits
        


In [46]:
question_caption_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
answer_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
answer_tokenizer.pad_token = answer_tokenizer.eos_token 
train_loader = DataLoader(pytorch_valid, batch_size=16, shuffle=True)
modelvqa = MultimodalVQAModel()
device = torch.device("cuda:0")
modelvqa.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
checkpoint_path="model_checkpointvizwiz.pth"

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
for item in pytorch_train:
    print(item)
    break

{'image': tensor([[[0.8471, 0.8431, 0.8275,  ..., 0.7098, 0.7020, 0.7059],
         [0.8353, 0.8275, 0.8157,  ..., 0.7137, 0.7098, 0.7098],
         [0.8157, 0.8078, 0.8039,  ..., 0.7137, 0.7098, 0.7098],
         ...,
         [0.1529, 0.1451, 0.1569,  ..., 0.8118, 0.8039, 0.7882],
         [0.1529, 0.1686, 0.1843,  ..., 0.8275, 0.8235, 0.8196],
         [0.2039, 0.2039, 0.1882,  ..., 0.8275, 0.8275, 0.8275]],

        [[0.3255, 0.3176, 0.3137,  ..., 0.6000, 0.6000, 0.6000],
         [0.3137, 0.3059, 0.3020,  ..., 0.6078, 0.6039, 0.5961],
         [0.3059, 0.2980, 0.2980,  ..., 0.6078, 0.6078, 0.6000],
         ...,
         [0.1569, 0.1412, 0.1490,  ..., 0.7255, 0.7176, 0.7059],
         [0.1608, 0.1647, 0.1725,  ..., 0.7412, 0.7333, 0.7294],
         [0.1922, 0.1961, 0.1765,  ..., 0.7412, 0.7373, 0.7333]],

        [[0.1294, 0.1373, 0.1333,  ..., 0.3255, 0.3176, 0.3255],
         [0.1255, 0.1255, 0.1255,  ..., 0.3216, 0.3137, 0.3176],
         [0.1255, 0.1176, 0.1216,  ..., 0.3176, 

In [32]:
for batch in train_loader:
    print(batch['captions'])
    break

[('a pair of scissors sitting on top of a bed', 'a television screen with a pizza in front of it', 'a computer monitor sitting on top of a wooden desk', 'a bottle of alcohol sitting on the side of a street', 'a bottle of water sitting on top of a table', 'a room with a window and a vase of flowers', 'a microwave oven sitting on top of a counter', 'a black and white photo of a computer screen', 'a brown teddy bear wearing a red bow tie', 'a blurry image of a sunset with a cloudy sky', 'a black dog laying on the ground', 'a person sitting in front of a box filled with food', 'a glass with a liquid inside of it', 'a person holding a purple frisbee in their hand', 'a bottle of beer sitting on top of a counter', 'a sign that says "don\'t drink" on it')]


In [47]:
def train(model, train_loader, optimizer, num_epochs, device, checkpoint_path="model_checkpointvizwiz.pth"):

    """
    Trains the MultimodalVQAModel.

    Arguments:
    - model (nn.Module): The multimodal VQA model to train.
    - train_loader (DataLoader): DataLoader providing batches of training data.
    - optimizer (torch.optim.Optimizer): Optimizer for updating model parameters.
    - num_epochs (int): Number of training epochs.
    - device (torch.device): Device (CPU or GPU) for computation.
    - checkpoint_path (str): Path to save model checkpoints.

    Returns:
    - None
    """
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            # Preprocess images, questions, and captions
            images = batch["image"].to(device)
            
            questions = question_caption_tokenizer(batch["question"], padding=True, return_tensors="pt").to(device)
            captions = [caption for group in batch["captions"] for caption in group]
            captions = question_caption_tokenizer(captions, padding=True, return_tensors="pt").to(device)
            answers = batch["answers"]  # Ensure this is in the right format as well

            # Tokenize answers
            answer_encodings = answer_tokenizer(answers, padding=True, return_tensors="pt").to(device)
            
            

            # Forward pass
            optimizer.zero_grad()
            loss, logits = model(
                image=images,
                questions = questions,
                captions = captions,
                answers= answers
            )

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Track loss
            total_loss += loss.item()
        
        # Average loss per epoch
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        # Save checkpoint every 10 epochs
        
        torch.save(model.state_dict(),checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch + 1}")

In [20]:
train(modelvqa,train_loader,optimizer, 10, device, checkpoint_path)

Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 924/924 [06:46<00:00,  2.27it/s]


Epoch 1/10, Loss: 8.9436
Checkpoint saved at epoch 1


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████| 924/924 [06:46<00:00,  2.27it/s]


Epoch 2/10, Loss: 8.9780
Checkpoint saved at epoch 2


Epoch 3: 100%|████████████████████████████████████████████████████████████████████████| 924/924 [06:46<00:00,  2.28it/s]


...

Epoch 8/10, Loss: 8.9747
Checkpoint saved at epoch 8


Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 924/924 [06:46<00:00,  2.28it/s]


Epoch 9/10, Loss: 8.9784
Checkpoint saved at epoch 9


Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 924/924 [06:47<00:00,  2.27it/s]


Epoch 10/10, Loss: 8.9512
Checkpoint saved at epoch 10


In [21]:
def validate(model, val_loader, device):
    """
    Validates the MultimodalVQAModel on the validation dataset.

    Parameters:
    - model (nn.Module): The trained multimodal VQA model to validate.
    - val_loader (DataLoader): DataLoader providing batches of validation data.
    - device (torch.device): Device (CPU or GPU) for computation.

    Returns:
    - accuracy (float): Validation accuracy as a fraction between 0 and 1.
    """
    
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            # Preprocess images, questions, captions, and answers
            images = batch["image"].to(device)
            questions = question_caption_tokenizer(batch["question"], padding=True, return_tensors="pt").to(device)
            captions = [caption for group in batch["captions"] for caption in group]
            captions = question_caption_tokenizer(captions, padding=True, return_tensors="pt").to(device)
            answers = batch["answers"]

            # Tokenize answers
            answer_encodings = answer_tokenizer(answers, padding=True, return_tensors="pt").to(device)

            # Forward pass to get logits
            _, logits = model(image=images, questions=questions, captions=captions, answers=answers)

            # Get predicted answers (highest logit)
            predicted_ids = torch.argmax(logits, dim=-1)
            predicted_answers = answer_tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
            print(predicted_answers)

            # Calculate accuracy by comparing predicted answers with ground-truth answers
            correct_predictions += sum([1 for pred, true in zip(predicted_answers, answers) if pred.strip().lower() == true.strip().lower()])
            total_predictions += len(answers)

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"Validation Accuracy: {accuracy:.4f}")
    return accuracy


In [22]:
valid_loader = DataLoader(pytorch_valid, batch_size=16, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
validate(modelvqa,valid_loader,device)


Validating:   1%|▋                                                                      | 1/103 [00:01<02:57,  1.74s/it]

[',.,, and of and', ',,,,,,,', ',,,,,,,', ',,,,,,\n', ',,,\n\n\n\n', ',,,,,,\n', ',,,\n\n\n\n', ',...,,,', ',,,,\n\n\n', ',,, and and and and', ',,. and and and and', ',..,,,,', ',,,,,,,', ',,,,\n\n\n', ',,,,,\n\n', ',,,,\n\n\n']


Validating:   2%|█▍                                                                     | 2/103 [00:03<03:19,  1.97s/it]

['\n,,,', '\n,,\n', ',,,,', ',,,,', ',,,\n', ",'s,\n", ',,,,', ',,,,', ',,,,', ',,,,', ',,,,', ',,,,', ',,..', ',,,,', ',,,,', ',,,,']


Validating:   3%|██                                                                     | 3/103 [00:05<03:07,  1.87s/it]

...

[',,,,\n\n', ',,,,..', ',,,\n\n\n', ',,,,,\n', ',,,,,,', ',,,,,\n', ',.,,,,', ',,..,\n', ',,.\n\n\n', '\n of..\n\n', ',,,,\n\n', ',,,,\n\n', ',,,,,,', ',,,.,\n', ',,,..\n', ',,,,,,']


Validating:  15%|██████████▏                                                           | 15/103 [00:30<03:11,  2.18s/it]

[',.....\n\n', ',,,, music and and and', ',,.... and and', ',,,,,,,,', ',,,,,\n\n\n', '\n,,\n\n\n\n\n', ',,,,,\n\n\n', ',,,.,,,,', ',......\n', ',,,,,\n\n\n', ',,..,,\n\n', ',,,\n\n\n\n\n', ', of.,,\n\n\n', ',..,\n\n\n\n', ',,,,,,,\n', ',,,,\n\n\n\n']


Validating:  15%|██████████▏                                                           | 15/103 [00:31<03:05,  2.11s/it]


KeyboardInterrupt: 